# Test

In [35]:
import warnings
warnings.filterwarnings("ignore")

# facebook_2020

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

import copy
import numpy as np
import torch
import tqdm
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim

# change this
dataset = 'facebook_2020' # folder and dataset name

cols = ['age', 'UCLA_3item_sum', 'stress', 'depression',
        'Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional_Stability', 'Openness_to_newexp'] # outcome columns

lr = 0.1

# START

train = pickle.load(open(dataset + '/BERTopic/run_1/train.pkl', 'rb'))
test = pickle.load(open(dataset + '/BERTopic/run_1/test.pkl', 'rb'))

temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
df = pd.read_csv(dataset + '.csv') # loading in actual dataset

df['isFemale'] = df['sex'] == 'Female'

def func(x):
    education = x['education']
    bachelor_plus = ["Master's degree or equivalent", "Professional degree (MD, JD, etc.), doctorate degree or higher"]
    bachelor_minus = ["High school diploma or equivalent", "Vocational/Technical Training"]

    if education in bachelor_plus:
        return 'bachelor+'
    elif education in bachelor_minus:
        return "bachelor-"
    else:
        return 'bachelor'

df['education_bucket'] = df.apply(lambda x : func(x), axis = 1)
label_encoder = LabelEncoder()
df['education_bucket'] = label_encoder.fit_transform(df['education_bucket'])

def func(x):
    income = x['income']
    under = ["$10,000 - $19,999", "$20,000 - $34,999", "$35,000 - $49,999"]
    over = ["$100,000 - $149,999", "More than $150,000"]

    if income in under:
        return '<50k+'
    elif income in over:
        return "100k+"
    else:
        return '$50,000 - $99,999'

df['income_bucket'] = df.apply(lambda x : func(x), axis = 1)
label_encoder = LabelEncoder()
df['income_bucket'] = label_encoder.fit_transform(df['income_bucket'])

label_encoder = LabelEncoder()
df['marital_status'] = label_encoder.fit_transform(df['marital_status'])

merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id_x', 'user_id', 'message_x'] + cols]
merged.columns = ['message_id', 'user_id', 'message'] + cols

user_ids = {user_id : user_data.index.tolist() for user_id, user_data in merged.groupby('user_id')} # gets all users and their corresponding indices in the df
all_user_ids = list(user_ids.keys())

labels_data = merged[['user_id'] + cols] # df of user_ids and all the labels to consider
user_ids_avg_dict = {} # populated to be user_id : average distribution

all_ys = []
for key in user_ids.keys():
    all_ys.append(labels_data[labels_data['user_id'] == key][cols].iloc[0])

for outcome in cols:
    print(outcome)
    print()

    y = [i[outcome] for i in all_ys]
    
    # REGRESSION

    all_scores = {}

    for model in ['GensimLDA', 'Mallet_LDA', 'CTM', 'BERTopic', 'NMF']:
        print(model)
        
        train_error_list = []
        test_error_list = []

        r2s_train = []
        r2s_test = []

        for run in range(1, 6):
            print("Run: " + str(run))
            print()

            topics = []
            with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
                reader = csv.reader(f)
                for row in reader:
                    topic_list = [item.strip() for item in row if item.strip()]
                    topics.append(topic_list)

            X = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_avg_topic_distribution.pkl', 'rb'))

            # 80-20 split --> didn't use train-test-split function since its already shuffled
            X_train = X[:round(0.80 * len(X))]
            X_test = X[round(0.80 * len(X)):]

            y_train = y[:round(0.80 * len(X))]
            y_test = y[round(0.80 * len(X)):]

            X_train = np.array(X_train)
            X_test = np.array(X_test)
            y_train = np.array(y_train)
            y_test = np.array(y_test)

            torch.manual_seed(42) # for reproducibility

            X_train_NN = torch.tensor(X_train, dtype = torch.float32)
            y_train_NN = torch.tensor(y_train, dtype = torch.float32).reshape(-1, 1)
            X_test_NN = torch.tensor(X_test, dtype = torch.float32)
            y_test_NN = torch.tensor(y_test, dtype = torch.float32).reshape(-1, 1)

            model_NN = nn.Sequential(
                nn.Linear(X_train_NN.shape[1], 1),
            )

            loss_fn = nn.MSELoss()
            optimizer = optim.Adam(model_NN.parameters(), lr = lr)

            n_epochs = 500
            batch_size = 64
            batch_start = torch.arange(0, len(X_train), batch_size)

            best_mse = np.inf
            best_weights = None
            history = []

            for epoch in range(n_epochs):
                model_NN.train()
                with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
                    bar.set_description(f"Epoch {epoch}")
                    for start in bar:

                        X_batch = X_train_NN[start : start+batch_size]
                        y_batch = y_train_NN[start : start+batch_size]

                        y_pred = model_NN(X_batch)
                        loss = loss_fn(y_pred, y_batch)

                        optimizer.zero_grad()
                        loss.backward()

                        optimizer.step()

                        bar.set_postfix(mse=float(loss))

                model_NN.eval()
                y_pred = model_NN(X_test_NN)
                mse = loss_fn(y_pred, y_test_NN)
                mse = float(mse)
                history.append(mse)
                if mse < best_mse:
                    best_mse = mse
                    best_weights = copy.deepcopy(model_NN.state_dict())

                if epoch % 50 == 0:
                    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

            model_NN.load_state_dict(best_weights)

            y_pred_train = model_NN(X_train_NN).detach().numpy()
            y_pred_test = model_NN(X_test_NN).detach().numpy()

            train_error = np.sqrt(mean_squared_error(y_pred_train, y_train))
            test_error = np.sqrt(mean_squared_error(y_pred_test, y_test))

            train_error_list.append(train_error)
            test_error_list.append(test_error)

            print(f'Train RMSE: {train_error}')
            print(f'Test RMSE: {test_error}')

            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)

            r2s_train.append(r2_train)
            r2s_test.append(r2_test)
            
            print(f'R2 Train: {r2_train}')
            print(f'R2 Test: {r2_test}')

            all_scores[model] = {
                'Train RMSE': train_error_list,
                'Test RMSE': test_error_list,
                'R2 Train': r2s_train,
                'R2 Test': r2s_test,

                'y_train': y_train,
                'y_train_pred': y_pred_train,
                'y_test': y_test,
                'y_test_pred': y_pred_test
            }
        
        print()

    print()

    for m in all_scores.keys():
        print(f'{m} Average Train RMSE: {np.mean(all_scores[m]["Train RMSE"])}')
        print(f'{m} Average Test RMSE: {np.mean(all_scores[m]["Test RMSE"])}')
        print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
        print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
        print()

    with open('all_results/' + dataset + '_all_scores_' + outcome + '_NN.pkl', 'wb') as f:
        pickle.dump(all_scores, f)

age

GensimLDA
Run: 1

Epoch 1, Loss: 1754.3682861328125
Epoch 51, Loss: 112.29078674316406
Epoch 101, Loss: 112.30257415771484
Epoch 151, Loss: 112.68301391601562
Epoch 201, Loss: 113.63865661621094
Epoch 251, Loss: 114.87398529052734
Epoch 301, Loss: 116.0321044921875
Epoch 351, Loss: 116.93688201904297
Epoch 401, Loss: 117.54356384277344
Epoch 451, Loss: 117.87154388427734
Train RMSE: 10.507967544140913
Test RMSE: 10.84224266615706
R2 Train: 0.20843269492981997
R2 Test: 0.18096409056151608
Run: 2

Epoch 1, Loss: 1754.3682861328125
Epoch 51, Loss: 112.29078674316406
Epoch 101, Loss: 112.30257415771484
Epoch 151, Loss: 112.68301391601562
Epoch 201, Loss: 113.63865661621094
Epoch 251, Loss: 114.87398529052734
Epoch 301, Loss: 116.0321044921875
Epoch 351, Loss: 116.93688201904297
Epoch 401, Loss: 117.54356384277344
Epoch 451, Loss: 117.87154388427734
Train RMSE: 10.507967544140913
Test RMSE: 10.84224266615706
R2 Train: 0.20843269492981997
R2 Test: 0.18096409056151608
Run: 3

Epoch 1, Lo

# Regression

In [49]:
import warnings
warnings.filterwarnings("ignore")

# facebook_2020

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# change this
dataset = 'facebook_2020' # folder and dataset name

cols = ['age', 'UCLA_3item_sum', 'stress', 'depression',
        'Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional_Stability', 'Openness_to_newexp'] # outcome columns

for outcome in cols:
    print(outcome)
    print()

    train = pickle.load(open(dataset + '/BERTopic/run_1/train.pkl', 'rb'))
    test = pickle.load(open(dataset + '/BERTopic/run_1/test.pkl', 'rb'))

    temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
    df = pd.read_csv(dataset + '.csv') # loading in actual dataset

    df['isFemale'] = df['sex'] == 'Female'

    def func(x):
        education = x['education']
        bachelor_plus = ["Master's degree or equivalent", "Professional degree (MD, JD, etc.), doctorate degree or higher"]
        bachelor_minus = ["High school diploma or equivalent", "Vocational/Technical Training"]

        if education in bachelor_plus:
            return 'bachelor+'
        elif education in bachelor_minus:
            return "bachelor-"
        else:
            return 'bachelor'

    df['education_bucket'] = df.apply(lambda x : func(x), axis = 1)
    label_encoder = LabelEncoder()
    df['education_bucket'] = label_encoder.fit_transform(df['education_bucket'])

    def func(x):
        income = x['income']
        under = ["$10,000 - $19,999", "$20,000 - $34,999", "$35,000 - $49,999"]
        over = ["$100,000 - $149,999", "More than $150,000"]

        if income in under:
            return '<50k+'
        elif income in over:
            return "100k+"
        else:
            return '$50,000 - $99,999'

    df['income_bucket'] = df.apply(lambda x : func(x), axis = 1)
    label_encoder = LabelEncoder()
    df['income_bucket'] = label_encoder.fit_transform(df['income_bucket'])

    label_encoder = LabelEncoder()
    df['marital_status'] = label_encoder.fit_transform(df['marital_status'])

    merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id_x', 'user_id', 'message_x'] + cols]
    merged.columns = ['message_id', 'user_id', 'message'] + cols

    user_ids = {user_id : user_data.index.tolist() for user_id, user_data in merged.groupby('user_id')} # gets all users and their corresponding indices in the df
    all_user_ids = list(user_ids.keys())

    labels_data = merged[['user_id'] + cols] # df of user_ids and all the labels to consider
    user_ids_avg_dict = {} # populated to be user_id : average distribution
    y = [] # the outcome we care about

    for key in user_ids.keys():
        y.append(labels_data[labels_data['user_id'] == key][outcome].iloc[0])

    # REGRESSION

    all_scores = {}

    for model in ['GensimLDA', 'Mallet_LDA', 'BERTopic', 'NMF', 'CTM']:
        print(model)
        
        train_error_list = []
        test_error_list = []

        r2s_train = []
        r2s_test = []

        for run in range(1, 6):
            print("Run: " + str(run))
            print()

            topics = []
            with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
                reader = csv.reader(f)
                for row in reader:
                    topic_list = [item.strip() for item in row if item.strip()]
                    topics.append(topic_list)

            X = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_avg_topic_distribution.pkl', 'rb'))

            # 80-20 split --> didn't use train-test-split function since its already shuffled
            X_train = X[:round(0.80 * len(X))]
            X_test = X[round(0.80 * len(X)):]

            y_train = y[:round(0.80 * len(X))]
            y_test = y[round(0.80 * len(X)):]

            X_train = np.array(X_train)
            X_test = np.array(X_test)
            y_train = np.array(y_train)
            y_test = np.array(y_test)

            lr = LinearRegression().fit(X_train, y_train)
            
            y_pred_train = lr.predict(X_train)
            y_pred_test = lr.predict(X_test)

            train_error = np.sqrt(mean_squared_error(y_pred_train, y_train))
            test_error = np.sqrt(mean_squared_error(y_pred_test, y_test))

            train_error_list.append(train_error)
            test_error_list.append(test_error)

            print(f'Train RMSE: {train_error}')
            print(f'Test RMSE: {test_error}')

            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)

            r2s_train.append(r2_train)
            r2s_test.append(r2_test)
            
            print(f'R2 Train: {r2_train}')
            print(f'R2 Test: {r2_test}')

            all_scores[model] = {
                'Train RMSE': train_error_list,
                'Test RMSE': test_error_list,
                'R2 Train': r2s_train,
                'R2 Test': r2s_test,

                'y_train': y_train,
                'y_train_pred': y_pred_train,
                'y_test': y_test,
                'y_test_pred': y_pred_test
            }
        
        print()

    print()

    for m in all_scores.keys():
        print(f'{m} Average Train RMSE: {np.mean(all_scores[m]["Train RMSE"])}')
        print(f'{m} Average Test RMSE: {np.mean(all_scores[m]["Test RMSE"])}')
        print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
        print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
        print()

    with open('all_results/' + dataset + '_all_scores_' + outcome + '.pkl', 'wb') as f:
        pickle.dump(all_scores, f)

age

GensimLDA
Run: 1

Train RMSE: 9.151412818922838
Test RMSE: 10.808739626343064
R2 Train: 0.39961933730010935
R2 Test: 0.18601798905532063
Run: 2

Train RMSE: 9.151412818922838
Test RMSE: 10.808739626343064
R2 Train: 0.39961933730010935
R2 Test: 0.18601798905532063
Run: 3

Train RMSE: 9.151412818922838
Test RMSE: 10.808739626343064
R2 Train: 0.39961933730010935
R2 Test: 0.18601798905532063
Run: 4

Train RMSE: 9.151412818922838
Test RMSE: 10.808739626343064
R2 Train: 0.39961933730010935
R2 Test: 0.18601798905532063
Run: 5

Train RMSE: 9.151412818922838
Test RMSE: 10.808739626343064
R2 Train: 0.39961933730010935
R2 Test: 0.18601798905532063

Mallet_LDA
Run: 1

Train RMSE: 11.165818661144902
Test RMSE: 12.186993375025711
R2 Train: 0.1062181254218314
R2 Test: -0.03480339957278589
Run: 2

Train RMSE: 11.119262384038457
Test RMSE: 13.2936821705826
R2 Train: 0.11365589748113492
R2 Test: -0.2312756174111712
Run: 3

Train RMSE: 11.231907798990019
Test RMSE: 13.188333748171553
R2 Train: 0.095

# Classification

In [73]:
import warnings
warnings.filterwarnings("ignore")

# facebook_2020

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LogisticRegression

# change this
dataset = 'facebook_2020' # folder and dataset name

cols = ['marital_status', 'isFemale', 'education_bucket', 'income_bucket']

for outcome in cols:
    print(outcome)
    print()

    train = pickle.load(open(dataset + '/BERTopic/run_1/train.pkl', 'rb'))
    test = pickle.load(open(dataset + '/BERTopic/run_1/test.pkl', 'rb'))

    temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
    df = pd.read_csv(dataset + '.csv') # loading in actual dataset

    df['isFemale'] = df['sex'] == 'Female'

    def func(x):
        education = x['education']
        bachelor_plus = ["Master's degree or equivalent", "Professional degree (MD, JD, etc.), doctorate degree or higher"]
        bachelor_minus = ["High school diploma or equivalent", "Vocational/Technical Training"]

        if education in bachelor_plus:
            return 'bachelor+'
        elif education in bachelor_minus:
            return "bachelor-"
        else:
            return 'bachelor'

    df['education_bucket'] = df.apply(lambda x : func(x), axis = 1)
    label_encoder = LabelEncoder()
    df['education_bucket'] = label_encoder.fit_transform(df['education_bucket'])

    def func(x):
        income = x['income']
        under = ["$10,000 - $19,999", "$20,000 - $34,999", "$35,000 - $49,999"]
        over = ["$100,000 - $149,999", "More than $150,000"]

        if income in under:
            return '<50k+'
        elif income in over:
            return "100k+"
        else:
            return '$50,000 - $99,999'

    df['income_bucket'] = df.apply(lambda x : func(x), axis = 1)
    label_encoder = LabelEncoder()
    df['income_bucket'] = label_encoder.fit_transform(df['income_bucket'])

    label_encoder = LabelEncoder()
    df['marital_status'] = label_encoder.fit_transform(df['marital_status'])

    merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id_x', 'user_id', 'message_x'] + cols]
    merged.columns = ['message_id', 'user_id', 'message'] + cols

    user_ids = {user_id : user_data.index.tolist() for user_id, user_data in merged.groupby('user_id')} # gets all users and their corresponding indices in the df
    all_user_ids = list(user_ids.keys())

    labels_data = merged[['user_id'] + cols] # df of user_ids and all the labels to consider
    user_ids_avg_dict = {} # populated to be user_id : average distribution
    y = [] # the outcome we care about

    for key in user_ids.keys():
        y.append(labels_data[labels_data['user_id'] == key][outcome].iloc[0])

    num_epochs = 500

    all_scores = {}

    lrs = {
        'education_bucket': {
            'GensimLDA': 0.05,
            'Mallet_LDA': 0.01,
            'CTM': 0.05,
            'BERTopic': 0.5,
            'NMF': 0.001
        },

        'income_bucket': {
            'GensimLDA': 0.05,
            'Mallet_LDA': 0.01,
            'CTM': 0.2,
            'BERTopic': 0.5,
            'NMF': 0.001
        },

        'isFemale': {
            'GensimLDA': 0.05,
            'Mallet_LDA': 0.1,
            'CTM': 0.5,
            'BERTopic': 0.5,
            'NMF': 0.001
        },

        'marital_status': {
            'GensimLDA': 0.05,
            'Mallet_LDA': 0.1,
            'CTM': 0.08,
            'BERTopic': 0.5,
            'NMF': 0.001
        }
    }

    for model in ['GensimLDA', 'Mallet_LDA', 'BERTopic', 'NMF', 'CTM']:
        print(model)

        train_error_list = []
        test_error_list = []

        r2s_train = []
        r2s_test = []

        lr = lrs[outcome][model]

        for run in range(1, 6):
            print("Run: " + str(run))
            print()

            topics = []
            with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
                reader = csv.reader(f)
                for row in reader:
                    topic_list = [item.strip() for item in row if item.strip()]
                    topics.append(topic_list)

            X = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_avg_topic_distribution.pkl', 'rb'))

            # 80-20 split --> didn't use train-test-split function since its already shuffled
            X_train = X[:round(0.80 * len(X))]
            X_test = X[round(0.80 * len(X)):]

            y_train = y[:round(0.80 * len(X))]
            y_test = y[round(0.80 * len(X)):]

            X_train = np.array(X_train)
            X_test = np.array(X_test)
            y_train = np.array(y_train)
            y_test = np.array(y_test)

            train_users = all_user_ids[:round(0.80 * len(X))]
            test_users = all_user_ids[round(0.80 * len(X)):]

            # Convert arrays to torch tensors
            X_train_tensor = torch.tensor(np.array(X_train).astype(np.float32))
            y_train_tensor = torch.tensor(np.array(y_train).astype(np.longlong))  # Use long for classification
            X_test_tensor = torch.tensor(np.array(X_test).astype(np.float32))
            y_test_tensor = torch.tensor(np.array(y_test).astype(np.longlong))

            # Create datasets and dataloaders
            train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
            test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

            train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False)
            test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

            # Define the model
            class LogisticRegressionModel(nn.Module):
                def __init__(self, input_size, num_classes):
                    super(LogisticRegressionModel, self).__init__()
                    self.layer1 = nn.Linear(input_size, num_classes)

                def forward(self, x):
                    return self.layer1(x)

            input_size = X_train.shape[1]
            num_classes = len(np.unique(y_train))  # Assuming y_train contains all classes
            logit_model = LogisticRegressionModel(input_size, num_classes)

            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()  # This includes softmax
            optimizer = optim.Adam(logit_model.parameters(), lr=lr)

            loss_values = []
            early_stopping_triggered = False

            # Train the model
            for epoch in range(num_epochs):
                epoch_loss = 0
                for inputs, targets in train_loader:
                    # Forward pass
                    outputs = logit_model(inputs)
                    loss = criterion(outputs, targets)
                    
                    # Backward and optimize
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0) 
                
                epoch_loss /= len(train_loader.dataset)
                loss_values.append(epoch_loss)

                if epoch >= 50:
                    # Calculate the percentage change in loss
                    loss_change = (loss_values[epoch - 50] - loss_values[epoch]) / loss_values[epoch - 50]
                    
                    # If change in loss is less than 0.5%, stop training
                    if abs(loss_change) < 0.005:
                        print(f'Early stopping at epoch {epoch+1} due to minimal loss improvement.')
                        early_stopping_triggered = True
                        break

                if (epoch+1) % 10 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

            # Predict on the test set
            logit_model.eval()  # Set the model to evaluation mode

            with torch.no_grad():
                y_pred_train = logit_model(X_train_tensor)
                _, predicted_train = torch.max(y_pred_train.data, 1)

                y_pred_test = logit_model(X_test_tensor)
                _, predicted_test = torch.max(y_pred_test.data, 1)


            train_error = 1 - accuracy_score(predicted_train, y_train)
            test_error = 1 - accuracy_score(predicted_test, y_test)

            train_error_list.append(train_error)
            test_error_list.append(test_error)
            
            print(f'Train Error: {train_error}')
            print(f'Test Error: {test_error}')

            print()

            r2_train = r2_score(y_train, predicted_train)
            r2_test = r2_score(y_test, predicted_test)

            r2s_train.append(r2_train)
            r2s_test.append(r2_test)
            
            print(f'R2 Train: {r2_train}')
            print(f'R2 Test: {r2_test}')

            all_scores[model] = {
                'Train Error': train_error_list,
                'Test Error': test_error_list,
                'R2 Train': r2s_train,
                'R2 Test': r2s_test,

                'y_train': y_train,
                'y_train_pred': predicted_train.numpy(),
                'y_test': y_test,
                'y_test_pred': predicted_test.numpy()
            }

        print()

    print()

    for m in all_scores.keys():
        print(f'{m} Average Train Error: {np.mean(all_scores[m]["Train Error"])}')
        print(f'{m} Average Test Error: {np.mean(all_scores[m]["Test Error"])}')
        print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
        print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
        print()

    with open('all_results/' + dataset + '_all_scores_' + outcome + '.pkl', 'wb') as f:
        pickle.dump(all_scores, f)

marital_status



In [ ]:
# Regression: age, loneliness, stress, depression, personality ones

# Classification: sex (2), marital_status (3), education (3), income (3)

# education
    ## Bachelors
        ### Bachelors

    ## Bachelors or higher
        ### Masters
        ### Professional degree

    ## Bachelors or lower
        ### High school
        ### Vocational

# marital_status
    ## Married
    ## Never married
    ## Divorced

# isFemale
    ## Female
    ## Not Female

# income
    ## <50k
    ## 50 - 100k
    ## 100k+